In [1]:
import gurobipy as gp 
from gurobipy import GRB

In [2]:
import numpy as np
import math
def randInstanceGen(n,m,J,U):
    # Reset seed for safety
    np.random.seed(0)
    # Set seed
    np.random.seed(61836)
    
    c = np.random.randint(1,U+1,(J,n))
    a = np.random.randint(1,U+1,(n,m))
    
    b = np.max(a, axis=0)
    
    for i in range(0, m):
        if math.ceil(np.sum(a[:,i])/2) > b[i]:
            b[i] = math.ceil(np.sum(a[:,i])/2)
    
    return c, a, b

In [196]:
c, a, b = randInstanceGen(10,1,2,40)
c[1] = c[1]*-1
print(c)
print("**")
print(a)
print("**")
print(b)

[[ 34   1  37  17   1  40   3  32  31   7]
 [-29 -22 -12 -40 -20  -2 -23  -7  -7 -31]]
**
[[10]
 [23]
 [ 6]
 [35]
 [24]
 [20]
 [16]
 [24]
 [15]
 [38]]
**
[106]


In [203]:
def rectangular(c,a,b):
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[0]
    cons = b.shape[0]
    

    Items=list(np.arange(n))
    Dimensions= list(np.arange(cons))

    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    NW = (lexConstraint,hold)
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    SE = (hold, lexConstraint)
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    found.append(NW)
    found.append(SE)
    
    #points in () , tuples
    #rectangles 2D lists of points 
    #5
    while len(rectangles) != 0:
        #6&7
        R = rectangles.pop(0)
     
        #8
        R2 = [(R[0][0] , (R[0][1]+R[1][1])/2.0), R[1] ] 
        
        
        #lexmin(z1,z2) MAKE SURE IN R2 (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1], name = 'top bound')
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.optimize()
        
        z1 = m.objVal
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        #9
        tempNW = (z1,hold) 
        #10/11/12
        if tempNW != R[1]:
            found.append(tempNW)
            rectangles.append([tempNW, R[1]])
        #13  
        R3 = [R[0], (tempNW[0] - 1, (R[0][1] + R[1][1])/2.0)]
        
               
               
        #lexmin(z2,z1) = z_opt    
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R3[1][0], name = 'right bound')
        #m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R2[0][1] , name = 'bottom bound')
        
        m.optimize()
        
        z1=m.objVal
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('right bound')
        m.remove(old)
        #old = m.getConstrByName('bottom bound')
        #m.remove(old)
        
        #14
        tempSE = (hold,z1)
        if tempSE != R[0]:
            found.append(tempSE)
            rectangles.append([R[0],tempSE])
    return found 
    
    
    
    
    
    

In [204]:
rectangular(c,a,b)

[(0.0, 0.0),
 (98.0, -135.0),
 (9.0, -73.0),
 (5.0, -65.0),
 (22.0, -105.0),
 (12.0, -96.0),
 (2.0, -42.0),
 (1.0, -22.0),
 (59.0, -122.0),
 (55.0, -114.0),
 (11.0, -76.0),
 (4.0, -45.0),
 (92.0, -126.0),
 (53.0, -111.0),
 (61.0, -123.0)]